<img src='Title.png'>


<p style="text-align: right;"><font face="Microsoft JhengHei"> 精算二部 決策支援系統科 黃喬敬 #2606</font> </p><br>







<p style="text-align: center;"> <font size=7% face="Microsoft JhengHei">  Data Visualization 資料觀察  </font>  </p>






<font size=4% face="Arial Black" color="GOLD">Shortcut :  </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[系統載入](#SYSTEM) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵觀察](#FEATURES) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵關係](#RELATION) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[缺失觀察](#MISSING) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵報告](#REPORT) </font>&nbsp;

<a id="SYSTEM"></a>

# 系統載入

接下來會使用各種方式進行資料觀察。

In [1]:
from IPython.core.display import display, HTML
import warnings
#warnings.simplefilter(action='ignore', category=UserWarning)
import numpy             as np
import pandas            as pd
import tables
import matplotlib
import pandas_profiling
#matplotlib.use('Agg')
%matplotlib inline  
import matplotlib.pyplot as plt
import seaborn as sns
import sys,os, csv, timeit
import missingno as msno
import lightgbm as lgb

pd.set_option('display.max_columns', 500)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = (15,15)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['font.family']='sans-serif' 
#sns.set(font_scale = 1,font ='Microsoft JhengHei')
#sns.set_style("bright",{"font.sans-serif":['JhengHei', 'Arial']})

from matplotlib.font_manager import FontProperties
myfont=FontProperties(fname=r'Microsoft JhengHei.ttf')
sns.set(font=myfont.get_name())

sns_palette = 'bright'#bright, Set2, Set1, Set3, Paired , pastel, hls, husl, muted,colorblind
sns.set_palette(sns_palette)
plt.style.use('seaborn')
print('python',sys.version)
print('numpy',np.__version__)
print('pandas',pd.__version__)
print('tables',tables.__version__)
print('seaborn',sns.__version__)
print('matplotlib',matplotlib.__version__)
print('missingno',msno.__version__)
print('pandas_profiling','1.4.1')
print('lightgbm',lgb.__version__)

python 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:54:13) [MSC v.1900 64 bit (AMD64)]
numpy 1.14.2
pandas 0.24.2
tables 3.4.4
seaborn 0.9.0
matplotlib 2.2.2
missingno 0.4.1
pandas_profiling 1.4.1
lightgbm 2.2.3


## 存儲讀取HDF設置

<font size=2% color=GREEN>【不需修改】</font> 建立儲存HDF與讀取HDF的方程式

In [ ]:
def dfs_to_hdf(TrainDF,TestDF, HDFname, TrainID_ = '',TestID_ =''):
    if type(TrainID_) == pd.core.series.Series and  type(TestID_) == pd.core.series.Series:
        TrainDF.insert(0, "_ID__", TrainID_) 
        TestDF.insert(0, "_ID__", TestID_) 
    TrainDF['_targetY__'] = TargetColumn
    TrainDF.to_hdf(r'_temp/'+HDFname+'.hdf','Train',mode='w')
    TestDF.to_hdf(r'_temp/'+HDFname+'.hdf','Test')
    TrainDF.drop(['_ID__','_targetY__'],axis=1,inplace=True)
    TestDF.drop(['_ID__'],axis=1,inplace=True)
    
def hdf_to_dfs(HDFname):
    train = pd.read_hdf(r'_temp/'+HDFname+'.hdf','Train')
    test =  pd.read_hdf(r'_temp/'+HDFname+'.hdf','Test')
    return train.drop(['_ID__','_targetY__'],axis=1), test.drop(['_ID__'],axis=1) ,train['_targetY__'].values[0], train['_ID__'], test['_ID__']

## 觀察基本資料
<font size=2% color=GREEN>【不需修改】</font>  讀取基礎檔案資料，同時使用簡易LGBM模型判讀重要欄位，分為類別型與數值型。

<font size=2% color=ORANGE>【需要注意】</font>  資料觀察時應就真實資料與商業分析角度觀察各欄位特徵，在此僅藉由模型挑選示範部份欄位，僅供參考。


In [ ]:
if not os.path.exists(r'Fig'):
    os.makedirs(r'Fig')

TrainData, TestData, targetY, trainid, testid = hdf_to_dfs('data_basic')


def Cate_cols(train_):
    train = train_.copy()
    cat_cols, cat_, num_, all_ = train.drop([targetY], axis=1).select_dtypes(
        include=['object']).columns, [], [], []
    X_train, y_train = train.drop([targetY], axis=1), train[targetY]

    for col in X_train.select_dtypes(include=['object']).columns:
        X_train[col], _ = pd.factorize(X_train[col])
    if y_train.dtype == 'object':
        y_train, _ = pd.factorize(train[col])

    lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False, feature_name=list(
        X_train.columns.values), categorical_feature=list(cat_cols))
    gbm = lgb.train({'learning_rate': 0.05, }, lgb_train, num_boost_round=100,)
    for term in [x for _, x in sorted(zip(gbm.feature_importance(), X_train.columns), reverse=True)]:
        cat_.append(term) if term in cat_cols else num_.append(term)
        all_.append(term)
    return cat_, num_, all_


cat_cols, num_cols, all_cols = Cate_cols(TrainData)

TrainData['_data'] = 'Train ('+str(TrainData.shape[0]) + ')'
TestData['_data'] = 'Test ('+str(TestData.shape[0]) + ')'
AllData = pd.concat([TrainData, TestData], axis=0).reset_index(drop=True)
TrainData.drop(['_data'], axis=1, inplace=True)
TestData.drop(['_data'], axis=1, inplace=True)


<font size=4% face="Arial Black" color="GOLD">Shortcut :  </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[系統載入](#SYSTEM) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵觀察](#FEATURES) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵關係](#RELATION) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[缺失觀察](#MISSING) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵報告](#REPORT) </font>&nbsp;

<a id="FEATURES"></a>

# 重要欄位觀察

由於欄位較多，故使用演算法挑選較重要欄位進行觀察。真實案例應逐一觀察各欄位情形。




## 數值型欄位觀察

<font size=2% color=ORANGE>【需要注意】</font>  應該在初始資料時即做此觀察。

<font size=2% color=ORANGE>【需要注意】</font>  應觀察最大值與最小值，如有過份異常值通常可以在此發現。例如，BMI 最大值超過數千，此為明顯異常。

<font size=2% color=YELLOWGREEN>【可以修改】</font>  列出各欄位值的特徵，分別列出train與test，應比較分佈是否有差異

In [ ]:
display(TrainData[num_cols[:10]].describe())
display(TestData[num_cols[:10]].describe())

## 目標欄位觀察


<font size=2% color=YELLOWGREEN>【可以修改】</font>  畫出目標欄位的分佈

In [ ]:


fig  = plt.figure(figsize=(15,15))
sns.set(font=myfont.get_name(),palette=sns_palette)

if AllData[targetY][TrainData.shape[0]:].isna().sum() == TestData.shape[0]:
    pic = sns.catplot(x=targetY, kind="count",  data=TrainData )
    pic.fig.suptitle('Distribution_of_Y : '+ targetY)
else:
    pic = sns.catplot(x=targetY, kind="count",  data=AllData , col='_data')
    pic.fig.suptitle('Distribution_of_Y : '+ targetY)
    

plt.subplots_adjust(top=0.85)
plt.show()
plt.savefig(r'Fig/01_Distribution_of_Target.png', format='png', dpi=300)

## 數值型欄位觀察


<font size=2% color=YELLOWGREEN>【可以修改】</font>  畫出各欄對應目標的分佈

In [ ]:
# sns.distplot(TrainData[num_cols[0]])


fig = plt.figure(figsize=(12, 12))
sns.set(font=myfont.get_name(), palette=sns_palette)
for i, col in enumerate(num_cols[:4]):
    plt.subplot(2, 2, i+1)
    plt.title('Distribution_of '+col)
    for l in AllData[targetY].unique():
        sns.distplot(AllData[AllData[targetY] == l]
                     [col].dropna(), label=targetY + '= ' + str(l))


plt.suptitle('Distribution_of_Important_Features')
plt.tight_layout(h_pad=2)
plt.subplots_adjust(top=0.93)
fig.legend([targetY + '= ' + str(l)
            for l in AllData[targetY].unique()], loc='upper right')
plt.show()
plt.savefig(r'Fig/01_Distribution_of_Features.png', format='png', dpi=300)

## 類別型欄位觀察


<font size=2% color=YELLOWGREEN>【可以修改】</font>  畫出類別型欄位對應目標的分佈

In [ ]:

def cat_display(data,col):
    fig  = plt.figure(figsize=(12,12))
    if data[col].nunique()>4:
        sns.set(font=myfont.get_name(),font_scale=1.5,palette=sns_palette)
        data['_'+col] = data[col]
        temp = data[col].value_counts().index.tolist()[:7] if data[col].nunique()>9 else data[col].value_counts().index.tolist()[:3]
        data.loc[~data[col].isin(temp),'_'+col]='_others_'
        pic = sns.catplot(x=targetY, kind="count",  data=data , col='_'+col,col_wrap=4,col_order=temp + ['_others_'])
        
    else:
        sns.set(font=myfont.get_name(),font_scale= 0.4 * data[col].nunique() ,palette=sns_palette)
        pic = sns.catplot(x=targetY, kind="count",  data=data , col=col,col_wrap=data[col].nunique(),col_order=sorted(data[col].dropna().astype(str).unique()))
    
    
    pic.fig.suptitle('Distribution_of '+ col)
    plt.subplots_adjust(top=0.85)
    plt.show()
    plt.savefig(r'Fig/01_Distribution_of_Categorical_'+col+'.png', format='png', dpi=300)
    

for col in cat_cols[:3]:
    cat_display(TrainData,col)


<font size=4% face="Arial Black" color="GOLD">Shortcut :  </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[系統載入](#SYSTEM) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵觀察](#FEATURES) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵關係](#RELATION) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[缺失觀察](#MISSING) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵報告](#REPORT) </font>&nbsp;

<a id="RELATION"></a>

# 特徵之間的分佈關係

<font size=2% color=YELLOWGREEN>【可以修改】</font>  畫出特徵之間的關係，類別型欄位與數值型欄位關係。

In [ ]:


No_Classes = TrainData[targetY].nunique()

if len(cat_cols) >=1:
    
    x_cols = [col for col in cat_cols if TrainData[col].nunique()<=5]
    sns.set(font=myfont.get_name(),font_scale=1.5,palette=sns_palette)
    fig,axes=plt.subplots(min(3,len(x_cols)),min(3,len(num_cols)),figsize=(16,16)) if No_Classes == 2 else plt.subplots(4,1,figsize=(16,8))

    
    for i in range(min(3,len(x_cols))):
        for j in range(min(3,len(num_cols))):
            
            sns.violinplot(x=x_cols[i], y=num_cols[j], data=TrainData, hue=targetY, 
                           split=True if No_Classes==2 else False,
                           ax=axes[i,j] if No_Classes==2 else axes[i*2+j])
    sns.set(font=myfont.get_name(),font_scale=1.2)
    plt.suptitle('Relation between Features')    
    plt.tight_layout(h_pad=2)
    plt.subplots_adjust(top=0.93) 
    plt.show()
    plt.savefig(r'Fig/02_Relation_between_Categorical_Features.png', format='png', dpi=300)




<font size=2% color=YELLOWGREEN>【可以修改】</font>  畫出特徵之間的關係，數值型欄位關係。

In [ ]:
No_Classes = TrainData[targetY].nunique()


grid = [(j,k) for j in range(4) for k in range(4) if j < k]

fig,axes=plt.subplots(2,3,figsize=(16,12))
sns.set(font=myfont.get_name(),font_scale=1.5,palette='bright')
for i,(j,k) in enumerate(grid):
    sns.scatterplot(x=num_cols[j], y=num_cols[k], data=TrainData, hue=targetY, 
                       ax=axes[i//3,i%3])
    

plt.suptitle('Relation between two Features')    
plt.tight_layout(h_pad=2)
plt.subplots_adjust(top=0.93)
#plt.gcf().set_size_inches(16, 12)
plt.show()
plt.savefig(r'Fig/02_Relation_between_Numeric_Features.png', format='png', dpi=300)

## 特徵相關性觀察

使用 Pearson's correlation coefficient 繪製特徵之間的關係

In [ ]:
if TrainData[targetY].dtype==('O'):
    TrainData[targetY],_ = pd.factorize(TrainData[targetY])  
sns.set(font=myfont.get_name())
temp = TrainData[[targetY]+num_cols[:15]].dropna().corr()
sns.clustermap(temp,annot=True)

plt.gcf().set_size_inches(15, 15)
plt.show()
fig.savefig(r'Fig/03_Correlation_Coefficient.png', format='png', dpi=300)


<font size=4% face="Arial Black" color="GOLD">Shortcut :  </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[系統載入](#SYSTEM) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵觀察](#FEATURES) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵關係](#RELATION) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[缺失觀察](#MISSING) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵報告](#REPORT) </font>&nbsp;

<a id="MISSING"></a>

## 缺失值觀察
以白色呈現各欄位缺失狀況

In [ ]:
sns.set(font=myfont.get_name())
temp = AllData.filter(all_cols[:30])
temp = msno.matrix(temp)
fig.savefig(r'Fig/04_Missing_Values.png', format='png', dpi=300)

## 缺失關係
繪出缺失狀況發生的關聯性

In [ ]:
sns.set(font=myfont.get_name())
temp = AllData.filter(all_cols[:30])
msno.dendrogram(temp)
fig.savefig(r'Fig/04_Relation_of_Missing.png', format='png', dpi=300)


<font size=4% face="Arial Black" color="GOLD">Shortcut :  </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[系統載入](#SYSTEM) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵觀察](#FEATURES) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵關係](#RELATION) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[缺失觀察](#MISSING) </font>&nbsp;
<font size=5% face="Microsoft JhengHei">[特徵報告](#REPORT) </font>&nbsp;

<a id="REPORT"></a>

# 特徵報告
使用統計分析等等，建構各欄特徵的統計初礎分析報告。

In [ ]:
%%javascript
require(
        ["notebook/js/outputarea"],
        function (oa) {
            oa.OutputArea.auto_scroll_threshold = -1;
            console.log("Setting auto_scroll_threshold to -1");
        });

In [ ]:
import warnings
warnings.filterwarnings('ignore')
pfr = pandas_profiling.ProfileReport(AllData)
pfr.to_file('Fig/Feautre_Report.html')
display(pfr)